In [1]:
import pandas as pd
import seaborn as sn
import numpy as np
import tensorflow as tf
import joblib
from sklearn.manifold import TSNE
import plotly.express as px


2024-04-10 13:52:11.110671: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 13:52:11.133027: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 13:52:11.133048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 13:52:11.133608: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 13:52:11.137401: I tensorflow/core/platform/cpu_feature_guar

In [2]:
ENCODER =joblib.load("../data/NCRD/ncrd95-uniprot-ma_clf_enc.joblib")
EPOCHS = 20
CNN_DIM = 70
CNNS_DIM = 50
GRU_DIM = 60
TRAIN = "../data/NCRD/TrainNcrd95-ma.csv"
TEST = "../data/NCRD/TestNcrd95-ma.csv"

MODELDIR = "../../models/Fase5-NCRD"


In [3]:
with tf.device("cpu"):
    train = pd.read_csv(TRAIN)
    train = train[["Class","Sequence"]]
    test = pd.read_csv(TEST)
    sequences = pd.concat([train,test], axis = 0).Sequence.apply(lambda x: " ".join(x))


2024-04-10 13:52:12.026663: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 13:52:12.043516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 13:52:12.043684: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
MAX_LEN = max(pd.concat([train[train.Class != "NonR"],test[test.Class != "NonR"]], axis = 0).Sequence.apply(lambda x: len(x)))
MAX_LEN

2975

In [5]:
len(ENCODER.categories_[0])

11

In [6]:
# test["Class_b"] = np.where(test.Class.to_numpy() == "NonR",0,1)

In [7]:
# test["Sequence"] = test.Sequence.apply(lambda x: " ".join(x))
# test.replace("-","Non antibiotic", inplace=True)



test["Sequence"] = test.Sequence.apply(lambda x: " ".join(x))
X_test = tf.convert_to_tensor(test.Sequence.to_list())
y_test = test.Class.to_numpy().reshape(-1,1)

2024-04-10 13:52:12.338363: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1101] failed to allocate 18.31MiB (19202048 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-04-10 13:52:12.338402: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1101] failed to allocate 16.48MiB (17282048 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [8]:
grur = tf.keras.models.Sequential()        
grur.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = MAX_LEN))
grur.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = GRU_DIM, input_length = MAX_LEN, mask_zero = True))
grur.add(tf.keras.layers.GRU(512,return_sequences=True))
grur.add(tf.keras.layers.GRU(512,return_sequences=True))
grur.add(tf.keras.layers.GlobalAveragePooling1D())
grur.layers[0].adapt(sequences)

In [9]:
cnnr = tf.keras.models.Sequential()        
cnnr.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = MAX_LEN))
cnnr.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = CNN_DIM, input_length = MAX_LEN, mask_zero = True))
cnnr.add(tf.keras.layers.Conv1D(filters = 512,   kernel_size = 5, activation = "relu"))
cnnr.add(tf.keras.layers.Conv1D(filters = 512/2, kernel_size = 5, activation = "relu"))
cnnr.add(tf.keras.layers.Conv1D(filters = 512/4, kernel_size = 5, activation = "relu"))
cnnr.add(tf.keras.layers.Conv1D(filters = 512/8, kernel_size = 5, activation = "relu"))
cnnr.add(tf.keras.layers.GlobalAveragePooling1D())
cnnr.layers[0].adapt(sequences)

In [10]:
cnnsr = tf.keras.models.Sequential()        
cnnsr.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = MAX_LEN))
cnnsr.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = GRU_DIM, input_length = MAX_LEN, mask_zero = True))
cnnsr.add(tf.keras.layers.Conv1D(filters = 512,   kernel_size = 5, activation = "relu"))
cnnsr.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 5, activation = "relu"))
cnnsr.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 5, activation = "relu"))
cnnsr.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 5, activation = "relu"))
cnnsr.add(tf.keras.layers.GlobalAveragePooling1D())
cnnsr.layers[0].adapt(sequences)

In [11]:
def model_apply(model,x):
    model.pop()
    maxpolling = model.predict(x)
    return maxpolling

In [12]:
tf.keras.backend.clear_session()
cnn = tf.keras.models.load_model(f"{MODELDIR}/cnn")
cnn_emb_fitted = model_apply(cnn,X_test)

tf.keras.backend.clear_session()
cnn_emb_random = cnnr.predict(X_test)

: 

In [ ]:
tf.keras.backend.clear_session()
cnns = tf.keras.models.load_model(f"{MODELDIR}/cnn-same")
cnns_emb_fitted = model_apply(cnns,X_test)

tf.keras.backend.clear_session()
cnns_emb_random = cnnsr.predict(X_test)

2024-04-10 13:50:40.006894: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 714.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-10 13:50:40.330773: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 714.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-10 13:50:40.672086: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 714.12MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-10 13:50:41.006944: W external/local_tsl/tsl/framewor

  8/288 [..............................] - ETA: 7s

2024-04-10 13:50:41.681019: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 713.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


288/288 [==============================] - 9s 29ms/step


In [ ]:
tf.keras.backend.clear_session()
gru = tf.keras.models.load_model(f"{MODELDIR}/gru")
gru_emb_fitted = model_apply(gru,X_test)

tf.keras.backend.clear_session()
gru_emb_random = grur.predict(X_test)

2024-04-10 13:50:59.817512: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 40 outputs. Output shapes may be inaccurate.
2024-04-10 13:50:59.908210: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 40 outputs. Output shapes may be inaccurate.
2024-04-10 13:50:59.979309: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 40 outputs. Output shapes may be inaccurate.
2024-04-10 13:50:59.983366: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond' has 4 outputs but the _output_shapes attribute specifies shapes for 40 outputs. Output shapes may be inaccurate.
2024-04-10 13:51:00.008772: W tensorflow/core/common_runtime/graph_constructor.cc:840] Node 'cond/while' has 13 outputs but the _ou

InternalError: Graph execution error:

Detected at node cond/CudnnRNNV3 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/tiago/anaconda3/envs/deepsea/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/tiago/anaconda3/envs/deepsea/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/tiago/anaconda3/envs/deepsea/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/tiago/anaconda3/envs/deepsea/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/tiago/anaconda3/envs/deepsea/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/home/tiago/anaconda3/envs/deepsea/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/home/tiago/anaconda3/envs/deepsea/lib/python3.11/asyncio/events.p

In [ ]:
test = test.replace({"macrolide-lincosamide-streptogramin":"MLS","NonR":"NRP"})

In [ ]:
cnn_fit_tsne  =  TSNE(n_components=2, learning_rate="auto", n_iter=1000, perplexity=30).fit_transform(cnn_emb_fitted)
cnn_rand_tsne =  TSNE(n_components=2, learning_rate="auto", n_iter=1000, perplexity=30).fit_transform(cnn_emb_random)

In [ ]:
category_orders = np.array(['MLS', 'aminoglycoside', 'beta-lactam', 'chloramphenicol','glycopeptide', 'macrolide', 'multidrug', 'phosphonic acid','rifamycin', 'tetracycline',"NRP"])

In [ ]:
fig = px.scatter(
    cnn_fit_tsne, x=0, y=1,
    color=test.Class, 
    category_orders = {"color": category_orders},
    labels={'color': 'Class'},
    title = "CNN HMD",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(legend_traceorder="normal")
fig.update_traces(marker_size=7)
fig.show()
fig.write_image(f"{MODELDIR}/emb/cnn-tsne-classes.svg")

In [ ]:
fig = px.scatter(
    cnn_rand_tsne, x=0, y=1,
    color=test.Class, 
    category_orders = {"color": category_orders},
    labels={'color': 'Class'},
    title = "CNN HMD",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(legend_traceorder="normal")
fig.update_traces(marker_size=7)
fig.show()
fig.write_image(f"{MODELDIR}/emb/cnn-random-tsne-classes.svg")

SAME

In [ ]:
cnns_fit_tsne  = TSNE(n_components=2, learning_rate="auto", n_iter=1000, perplexity=30).fit_transform(cnns_emb_fitted)
cnns_rand_tsne = TSNE(n_components=2, learning_rate="auto", n_iter=1000, perplexity=30).fit_transform(cnns_emb_random)

In [ ]:
fig = px.scatter(
    cnns_fit_tsne, x=0, y=1,
    color=test.Class, 
    category_orders = {"color": category_orders},
    labels={'color': 'Class'},
    title = "CNN-same HMD",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(legend_traceorder="normal")
fig.update_traces(marker_size=7)
fig.show()
fig.write_image(f"{MODELDIR}/emb/cnn-same-tsne-classes.svg")

In [ ]:
fig = px.scatter(
    cnns_rand_tsne, x=0, y=1,
    color=test.Class, 
    category_orders = {"color": category_orders},
    labels={'color': 'Class'},
    title = "CNN-same HMD",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(legend_traceorder="normal")
fig.update_traces(marker_size=7)
fig.show()
fig.write_image(f"{MODELDIR}/emb/cnn-same-random-tsne-classes.svg")

GRU

In [ ]:
gru_fit_tsne  =  TSNE(n_components=2, learning_rate="auto", n_iter=1000, perplexity=30).fit_transform(gru_emb_fitted)
gru_rand_tsne =  TSNE(n_components=2, learning_rate="auto", n_iter=1000, perplexity=30).fit_transform(gru_emb_random)

In [ ]:
fig = px.scatter(
    gru_fit_tsne, x=0, y=1,
    color=test.Class, 
    category_orders = {"color": category_orders},
    labels={'color': 'Class'},
    title = "GRU HMD",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(legend_traceorder="normal")
fig.update_traces(marker_size=7)
fig.show()
fig.write_image(f"{MODELDIR}/emb/gru-tsne-classes.svg")

In [ ]:
fig = px.scatter(
    gru_rand_tsne, x=0, y=1,
    color=test.Class, 
    category_orders = {"color": category_orders},
    labels={'color': 'Class'},
    title = "GRU HMD",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig.update_layout(legend_traceorder="normal")
fig.update_traces(marker_size=7)
fig.show()
fig.write_image(f"{MODELDIR}/emb/gru-random-tsne-classes.svg")

In [ ]:
tsne_df = pd.DataFrame({
    "CNN-1":cnn_fit_tsne[:,0],
    "CNN-2":cnn_fit_tsne[:,1],

    "CNN-Rand-1":cnn_rand_tsne[:,0],
    "CNN-Rand-2":cnn_rand_tsne[:,1],

    "CNNS-1":cnns_fit_tsne[:,0],
    "CNNS-2":cnns_fit_tsne[:,1],

    "CNNS-Rand-1":cnns_rand_tsne[:,0],
    "CNNS-Rand-2":cnns_rand_tsne[:,1],

    "GRU-1":gru_fit_tsne[:,0],
    "GRU-2":gru_fit_tsne[:,1],
    
    "GRU-Rand-1":gru_rand_tsne[:,0],
    "GRU-Rand-2":gru_rand_tsne[:,1],

    "Class":test.Class.to_list(),
    "Mech": test.Mech.to_list(),
})

In [ ]:
my_pallete = {cls:cor for cls,cor in zip(sorted(ENCODER.categories_[0]),sn.color_palette("pastel"))}
my_pallete["NonR"] = (0,0,0)
my_pallete["tetracycline"] = (211/255, 211/255, 211/255)

In [ ]:
figure, axes = plt.subplots(2, 3, figsize=(12,5), constrained_layout=True)
figure.suptitle("Resistance Classes")

sn.scatterplot(ax = axes[0,0], data = tsne_df, x = "CNN-1", y = "CNN-2", hue = "Class", palette = my_pallete, legend=False )
sn.scatterplot(ax = axes[1,0], data = tsne_df, x = "CNN-Rand-1", y = "CNN-Rand-2", hue = "Class", palette = my_pallete, legend=False )

sn.scatterplot(ax = axes[0,1], data = tsne_df, x = "CNNS-1", y = "CNNS-2", hue = "Class", palette = my_pallete, legend = False)
sn.scatterplot(ax = axes[1,1], data = tsne_df, x = "CNNS-Rand-1", y = "CNNS-Rand-2", hue = "Class", palette = my_pallete, legend = False)


sn.scatterplot(ax = axes[0,2], data = tsne_df, x = "GRU-1", y = "GRU-2", hue = "Class", palette = my_pallete, legend=False)
sn.scatterplot(ax = axes[1,2], data = tsne_df, x = "GRU-Rand-1", y = "GRU-Rand-2", hue = "Class", palette = my_pallete)
plt.legend(loc=(1.05, 0))


plt.savefig(f"{MODELDIR}/emb/tsne-align-classes.svg", format = "SVG", bbox_inches='tight')
plt.savefig(f"{MODELDIR}/emb/tsne-align-classes.png", format = "PNG", bbox_inches='tight')


In [ ]:
mechs = sorted(test.Mech.fillna("unknow").unique())
mechs

In [ ]:
my_pallete = {cls:cor for cls,cor in zip(mechs,sn.color_palette("pastel"))}
my_pallete["Non antibiotic"] = (0,0,0)

In [ ]:
figure, axes = plt.subplots(2, 3, figsize=(12,5), constrained_layout=True)
figure.suptitle("Resistance Mechanisms")

sn.scatterplot(ax = axes[0,0], data = tsne_df, x = "CNN-1", y = "CNN-2", hue = "Mech", palette = my_pallete, legend=False )
sn.scatterplot(ax = axes[1,0], data = tsne_df, x = "CNN-Rand-1", y = "CNN-Rand-2", hue = "Mech", palette = my_pallete, legend=False )

sn.scatterplot(ax = axes[0,1], data = tsne_df, x = "CNNS-1", y = "CNNS-2", hue = "Mech", palette = my_pallete, legend=False)
sn.scatterplot(ax = axes[1,1], data = tsne_df, x = "CNNS-Rand-1", y = "CNNS-Rand-2", hue = "Mech", palette = my_pallete, legend=False)

sn.scatterplot(ax = axes[0,2], data = tsne_df, x = "GRU-1", y = "GRU-2", hue = "Mech", palette = my_pallete, legend=False)
sn.scatterplot(ax = axes[1,2], data = tsne_df, x = "GRU-Rand-1", y = "GRU-Rand-2", hue = "Mech", palette = my_pallete)
plt.legend(loc=(1.05, 0))



plt.savefig(f"{MODELDIR}/emb/tsne-align-mechs.svg", format = "SVG", bbox_inches='tight')
plt.savefig(f"{MODELDIR}/emb/tsne-align-mechs.png", format = "PNG", bbox_inches='tight')